In [3]:
import pandas as pd
import numpy as np

# ==========================================
# 1. LOAD THE DATA
# ==========================================
print("--- 📥 STEP 1: LOADING DATA STREAMS ---")

# Load the "Truth" (Facebook Ads Export)
try:
    df_fb = pd.read_csv('facebook_ads_export.csv')
    print(f"✅ Loaded Facebook Data: {len(df_fb)} rows")
except FileNotFoundError:
    print("❌ Error: 'facebook_ads_export.csv' not found. Please upload the file.")

# Load the "Ledger" (Xero Purchases Export)
# FIX: Added 'header=4' because Xero exports have 4 lines of titles at the top.
try:
    df_xero = pd.read_csv('xero_purchases.csv', header=4)
    print(f"✅ Loaded Xero Data: {len(df_xero)} rows")
except FileNotFoundError:
    print("❌ Error: 'xero_purchases.csv' not found. Please upload the file.")

# ==========================================
# 2. DATA CLEANING & STANDARDIZATION
# ==========================================
print("\n--- 🧹 STEP 2: CLEANING DATA ---")

# Clean column names (remove spaces and special chars)
df_fb.columns = df_fb.columns.str.strip()
df_xero.columns = df_xero.columns.str.strip()

# DEBUG: Print column names to check if 'Reference' exists now
print("Xero Columns Found:", df_xero.columns.tolist())

# Ensure Transaction IDs are strings for matching
# For Facebook
if 'Transaction_ID' in df_fb.columns:
    df_fb['Transaction_ID'] = df_fb['Transaction_ID'].astype(str).str.strip()

# For Xero (Fixing the KeyError)
if 'Reference' in df_xero.columns:
    df_xero['Reference'] = df_xero['Reference'].astype(str).str.strip()
else:
    print("🚨 CRITICAL ERROR: 'Reference' column still missing. Check the CSV structure.")

# Ensure Amounts are numeric (remove currency symbols if present)
# FIX: Added 'r' before the regex pattern r'[^\d.]' to fix SyntaxWarning
# For Facebook: 'Amount_GBP'
if 'Amount_GBP' in df_fb.columns:
    df_fb['Amount_GBP'] = df_fb['Amount_GBP'].astype(str).replace(r'[^\d.]', '', regex=True)
    df_fb['Amount_GBP'] = pd.to_numeric(df_fb['Amount_GBP'], errors='coerce').fillna(0)

# For Xero: 'Debit' is the expense amount
if 'Debit' in df_xero.columns:
    df_xero['Debit'] = df_xero['Debit'].astype(str).replace(r'[^\d.]', '', regex=True)
    df_xero['Debit'] = pd.to_numeric(df_xero['Debit'], errors='coerce').fillna(0)

# ==========================================
# 3. THE MATCHING ENGINE
# ==========================================
print("\n--- 🔍 STEP 3: RECONCILING MEDIA SPEND ---")

# Merge Facebook Data (Left) with Xero Data (Right)
merged = pd.merge(df_fb, df_xero,
                  left_on='Transaction_ID',
                  right_on='Reference',
                  how='left')

# Calculate Variance: Xero Debit - Facebook Spend
merged['Xero_Amount'] = merged['Debit'].fillna(0)
merged['Variance'] = merged['Xero_Amount'] - merged['Amount_GBP']

# ==========================================
# 4. EXCEPTION REPORT & STATUS FLAGGING
# ==========================================
def flag_status(row):
    if pd.isna(row['Reference']):
        return "🚨 CRITICAL: Spent on FB but MISSING in Xero"
    elif abs(row['Variance']) > 0.01:
        return f"⚠️ VARIANCE: Xero differs by £{row['Variance']:.2f}"
    else:
        return "✅ RECONCILED"

merged['Status'] = merged.apply(flag_status, axis=1)

# Select and Rename columns for the final executive report
# We check if columns exist before selecting to avoid errors
cols_to_show = ['Transaction_ID', 'Amount_GBP', 'Xero_Amount', 'Status']
if 'Campaign_Name' in merged.columns:
    cols_to_show.insert(1, 'Campaign_Name')

print("\n--- 📊 MEDIA SPEND CONTROL REPORT ---")
print(merged[cols_to_show].to_markdown(index=False))

# Export to CSV for download
merged.to_csv('Media_Spend_Exception_Report.csv', index=False)
print("\n✅ Output saved: 'Media_Spend_Exception_Report.csv'")

--- 📥 STEP 1: LOADING DATA STREAMS ---
✅ Loaded Facebook Data: 7 rows
✅ Loaded Xero Data: 82 rows

--- 🧹 STEP 2: CLEANING DATA ---
Xero Columns Found: ['Date', 'Source', 'Description', 'Invoice Number', 'Reference', 'Debit', 'Credit', 'Running Balance', 'Gross', 'Net', 'VAT', 'Account Code', 'Account Type']

--- 🔍 STEP 3: RECONCILING MEDIA SPEND ---

--- 📊 MEDIA SPEND CONTROL REPORT ---
| Transaction_ID   | Campaign_Name               |   Amount_GBP |   Xero_Amount | Status                                       |
|:-----------------|:----------------------------|-------------:|--------------:|:---------------------------------------------|
| FB-2025-881      | Spring Sale - Ridgeway      |         1500 |          1500 | ✅ RECONCILED                                |
| FB-2025-882      | Brand Awareness - Rex       |          450 |           450 | ✅ RECONCILED                                |
| FB-2025-883      | Lead Gen - Hamilton         |         2100 |          2100 | ✅ RECONCILED  